In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### local
#root_path = '../data/open'

### colab
root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

In [ ]:
import pandas as pd
df1 = pd.read_parquet(f'{root_path}/train/1.회원정보/201807_train_회원정보.parquet')

In [ ]:
df2 = pd.read_parquet(f'{root_path}/train/1.회원정보/201808_train_회원정보.parquet')

In [ ]:
df3 = pd.read_parquet(f'{root_path}/train/1.회원정보/201809_train_회원정보.parquet')

In [ ]:
df4 = pd.read_parquet(f'{root_path}/train/1.회원정보/201810_train_회원정보.parquet')

In [ ]:
df5 = pd.read_parquet(f'{root_path}/train/1.회원정보/201811_train_회원정보.parquet')

In [ ]:
df6 = pd.read_parquet(f'{root_path}/train/1.회원정보/201812_train_회원정보.parquet')

In [ ]:
# df1 ~ df6을 리스트에 담는다
month_list = [df1, df2, df3, df4, df5, df6]

# pd.concat으로 수직 결합 (행 기준)
month = pd.concat(month_list, ignore_index=True)

# 결과 확인
print(month.shape)
month.head()

In [ ]:
month.duplicated(subset=['ID', '기준년월']).sum()

In [ ]:
# month DataFrame을 12조 파이널프로젝트 폴더에 저장
month.to_parquet(f'{root_path}/train_회원정보.parquet', index=False)

In [ ]:
# 파일 불러와서 customer로 이름 지정
customer = pd.read_parquet(f'{root_path}/train_회원정보.parquet')

In [ ]:
customer.iloc[:, :20].head()

In [ ]:
customer.iloc[:, 20:40].head()

In [ ]:
customer.iloc[:, 40:60].head()

In [ ]:
customer.iloc[:, 60:77].head()

In [ ]:
# 각 열별 결측치 개수 계산
missing_counts = customer.isnull().sum()

# 결측치가 1개라도 있는 열만 필터링
missing_counts = missing_counts[missing_counts > 0]

# 내림차순 정렬해서 보기 좋게 출력
missing_counts = missing_counts.sort_values(ascending=False)

# 결과 출력
print("📌 결측치가 있는 열과 그 개수:")
print(missing_counts)


In [ ]:
missing_ratio = (customer.isnull().mean() * 100).sort_values(ascending=False)
missing_ratio = missing_ratio[missing_ratio > 0]

print("📊 결측치 비율 (%):")
print(missing_ratio.round(2))


In [ ]:
##### 1. 결측치 처리

# '_1순위신용체크구분', '_2순위신용체크구분' → 결측치 '기타'로 채우기 및 인코딩
customer['_1순위신용체크구분'] = customer['_1순위신용체크구분'].fillna('기타')
customer['_2순위신용체크구분'] = customer['_2순위신용체크구분'].fillna('기타')
mapping = {'신용': 1, '체크': 0, '기타': -1}
customer['1순위신용체크구분_인코딩'] = customer['_1순위신용체크구분'].map(mapping)
customer['2순위신용체크구분_인코딩'] = customer['_2순위신용체크구분'].map(mapping)
customer.drop(columns=['_1순위신용체크구분','_2순위신용체크구분'], inplace=True)

# 가입통신회사코드 → 가입통신회사_S사여부
customer['가입통신회사_S사여부'] = (customer['가입통신회사코드'] == 'S사').astype(int)
customer.drop(columns=['가입통신회사코드'], inplace=True)

# 직장시도명 → 직장_수도권여부
customer['직장_수도권여부'] = customer['직장시도명'].isin(['서울', '경기']).astype(int)
customer.drop(columns=['직장시도명'], inplace=True)

# 결측치 많은 변수 삭제
customer.drop(columns=['최종카드발급일자','최종유효년월_신용_이용가능','최종유효년월_신용_이용'], inplace=True)

##### 2. 자료형 변환 및 파생변수 생성

# 거주시도명 → 거주지_수도권여부
customer['거주지_수도권여부'] = customer['거주시도명'].isin(['서울', '경기']).astype(int)
customer.drop(columns=['거주시도명'], inplace=True)

# 연회비발생카드수_B0M → 이진변수화
customer['연회비발생카드수_B0M_이진'] = customer['연회비발생카드수_B0M'].isin(['1개이상']).astype(int)
customer.drop(columns=['연회비발생카드수_B0M'], inplace=True)

# 데이터가 모두 동일한 변수 제거
columns_drop = ['상품관련면제카드수_B0M','임직원면제카드수_B0M','우수회원면제카드수_B0M','기타면제카드수_B0M']
customer.drop(columns=columns_drop, inplace=True)

# Life_Stage → 파생변수 생성 후 삭제
customer['Life_Stage_자녀성장_여부'] = customer['Life_Stage'].isin(['자녀성장(1)', '자녀성장(2)']).astype(int)
customer.drop(columns=['Life_Stage'], inplace=True)

# 연령: 숫자만 추출 → int형 변환
customer['연령'] = customer['연령'].str.extract(r'(\d+)').astype(float).astype('Int64')

##### 3. 상관관계 고려 변수 제거
columns_drop = ['이용금액_R3M_체크_가족','연회비할인카드수_B0M','할인금액_기본연회비_B0M','할인금액_제휴연회비_B0M']
customer.drop(columns=columns_drop, inplace=True)

##### 4. 유사/중복 의미 변수 제거
columns_drop = ['입회일자_신용','이용카드수_체크_가족','청구금액_기본연회비_B0M','청구금액_제휴연회비_B0M']
customer.drop(columns=columns_drop, inplace=True)


In [ ]:
# int64 → int32 (메모리 절약용)
for col in customer.select_dtypes(include='int64').columns:
    if customer[col].max() < 2_147_483_647:
        customer[col] = customer[col].astype('int32')

# float64 → float32 (메모리 절약용)
for col in customer.select_dtypes(include='float64').columns:
    customer[col] = customer[col].astype('float32')

# Parquet 파일로 저장
customer.to_parquet(f'{root_path}/cleaned_회원정보.parquet')


In [ ]:
import pandas as pd
tt = pd.read_parquet(f'{root_path}/cleaned_회원정보.parquet')
tt